Import library, buat koneksi dan function

In [5]:
import mysql.connector
import pandas as pd

# CREATE CONNECTION
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='Password_untuk_SQL',
    database='world')

# CREATE ACCES TO DATABASE
mycursor = mydb.cursor()

# QUERY
query='SELECT * FROM country;'

# EXECUTE QUERY
mycursor.execute(query)

# SAVE RESULT
result = mycursor.fetchall()

In [6]:
kursor = mydb.cursor()
def sql(query) :
    kursor.execute(query)
    hasil = kursor.fetchall()
    return pd.DataFrame(data = hasil, columns = kursor.column_names)
 

Tampilkan 10 nama negara (Name) dan jumlah bahasa (language) baik yang official maupun tidak di tiap negara, dari tabel Country dan CountryLanguage! Tampilkan hanya negara yang jumlah bahasanya di atas rata-rata jumlah bahasa semua negara! Urutkan dari negara yang jumlah bahasanya paling banyak!

,Country_Name,Total_Language
0,United States,12
1,Russian Federation,12
2,India,12
3,China,12
4,Canada,12
5,South Africa,11
6,Tanzania,11
7,Nigeria,10
8,Mozambique,10
9,Iran,10


In [3]:
sql('''
with negara_bahasa AS (
    select country.name as negara, count(language) as jumlah_bahasa
    from country join countrylanguage on country.code=countrylanguage.countrycode
    group by negara
),
rata_bahasa as(
    select avg(jumlah_bahasa) as rata_jumlah from negara_bahasa
)
select *
from negara_bahasa
where jumlah_bahasa > (select rata_jumlah from rata_bahasa)
order by jumlah_bahasa desc
limit 10;
''')


,negara,jumlah_bahasa
0,United States,12
1,Russian Federation,12
2,India,12
3,China,12
4,Canada,12
5,South Africa,11
6,Tanzania,11
7,Nigeria,10
8,Mozambique,10
9,Iran,10


Tampilkan 10 nama negara (Name), bentuk pemerintahan (GovernmentForm), jumlah population semua negara (Population), persentase populasi tiap negara, serta nomor index baris tiap negara dari tabel Country!

,Name,GovernmentForm,World_Population,Percentage,Row_Index
0,China,People'sRepublic,6078749450,21.0168,42
1,India,Federal Republic,6078749450,16.6755,100
2,United States,Federal Republic,6078749450,4.5792,224
3,Indonesia,Republic,6078749450,3.4893,99
4,Brazil,Federal Republic,6078749450,2.7985,31
5,Pakistan,Republic,6078749450,2.5743,165
6,Russian Federation,Federal Republic,6078749450,2.4172,182
7,Bangladesh,Republic,6078749450,2.1247,22
8,Japan,Constitutional Monarchy,6078749450,2.0845,110
9,Nigeria,Federal Republic,6078749450,1.8344,156


In [48]:
sql('''
SELECT name, GovernmentForm, 
SUM(population) OVER() AS world_population,
population*100/SUM(population) OVER() AS percentage,
ROW_NUMBER() OVER() AS row_index
FROM country
ORDER BY percentage DESC
LIMIT 10
''')

,name,GovernmentForm,world_population,percentage,row_index
0,China,People'sRepublic,6078749450,21.0168,42
1,India,Federal Republic,6078749450,16.6755,100
2,United States,Federal Republic,6078749450,4.5792,224
3,Indonesia,Republic,6078749450,3.4893,99
4,Brazil,Federal Republic,6078749450,2.7985,31
5,Pakistan,Republic,6078749450,2.5743,165
6,Russian Federation,Federal Republic,6078749450,2.4172,182
7,Bangladesh,Republic,6078749450,2.1247,22
8,Japan,Constitutional Monarchy,6078749450,2.0845,110
9,Nigeria,Federal Republic,6078749450,1.8344,156


Tampilkan jumlah kota di tiap kawasan regional (Region)! Tampilkan hanya kawasan di Benua Asia dan Eropa, serta tambahkan nomor sesuai jumlah kawasan regional di tiap benua (Continent)! Gunakan tabel City dan Country! Perhatikan contoh hasilnya seperti di bawah ini!

,Continent,Region,Number_of_City,Row_Group
0,Asia,Southern and Central Asia,555,1
1,Asia,Middle East,174,2
2,Asia,Southeast Asia,297,3
3,Asia,Eastern Asia,740,4
4,Europe,Southern Europe,156,1
5,Europe,Western Europe,186,2
6,Europe,Eastern Europe,371,3
7,Europe,Nordic Countries,35,4
8,Europe,Baltic Countries,10,5
9,Europe,British Islands,83,6


In [104]:
sql('''
WITH tabel1 AS (SELECT country.continent, country.region,
COUNT(city.name) AS number_of_city 
FROM country JOIN city ON country.code = city.countrycode
GROUP BY country.region)
SELECT *, ROW_NUMBER() OVER(PARTITION BY continent) AS row_group FROM tabel1
WHERE continent='Asia' OR continent='Europe'
ORDER BY continent
LIMIT 10
''')

,continent,region,number_of_city,row_group
0,Asia,Southern and Central Asia,555,1
1,Asia,Middle East,174,2
2,Asia,Southeast Asia,297,3
3,Asia,Eastern Asia,740,4
4,Europe,Southern Europe,156,1
5,Europe,Western Europe,186,2
6,Europe,Eastern Europe,371,3
7,Europe,Nordic Countries,35,4
8,Europe,Baltic Countries,10,5
9,Europe,British Islands,83,6


Segmen konsumen yang potensial biasanya banyak tinggal di daerah ibukota negara. Meskipun, tidak menutup kemungkinan kota di luar ibukota juga mengalami perkembangan secara pesat, baik secara kuantitas pasar maupun daya beli konsumen.  

GNP suatu negara tidak sedikit yang disumbang perusahaan-perusahaan yang beroperasi di daerah ibukota. Terkadang pabriknya di luar ibukota, namun kantor pusatnya akan ditempatkan di ibukota. Sehingga, warga ibukota merupakan pasar potensial, karena banyak dari mereka yang bekerja di perusahaan-perusahaan besar penyumbang GNP nasional. Maka, GNP kurang lebih juga mencerminkan daya beli warga di ibukota suatu negara.  

Untuk menyusun strategi global, sebuah multinational company membutuhkan data peringkat benua berdasarkan jumlah populasi warga ibukota dan peringkat benua berdasarkan rata-rata GNP semua negara di benua tersebut.  

Tulis query SQL dari tabel yang tersedia database World untuk mendapatkan output tabel seperti di bawah ini! Menurut analisis Anda, benua manakah yang perlu diprioritaskan untuk pemasaran produk perusahaan?  

,Continent,Total_Capital_Population,Average_GNP,Rank_Population,Rank_GNP
0,Asia,101322793,150105.725490,1,3
1,Europe,51917214,206497.065217,2,2
2,Africa,47714282,10182.017544,3,6
3,South America,28900527,107991.000000,4,4
4,North America,19120216,261854.789189,5,1
5,Oceania,1142110,15547.211111,6,5


In [170]:
sql('''
WITH tabel1 AS (
    SELECT country.continent, SUM(city.population) AS total_capital_population, 
    AVG(country.gnp) AS avg_gnp FROM country 
    JOIN city ON country.capital=city.id GROUP BY country.continent)
SELECT *, 
RANK() OVER(ORDER BY total_capital_population DESC) AS Rank_population,
RANK() OVER(ORDER BY avg_gnp DESC) AS Rank_GNP
FROM tabel1
ORDER BY rank_population
''')

,continent,total_capital_population,avg_gnp,Rank_population,Rank_GNP
0,Asia,101322793,150105.725490,1,3
1,Europe,51917214,206497.065217,2,2
2,Africa,47714282,10182.017544,3,6
3,South America,28900527,107991.000000,4,4
4,North America,19120216,261854.789189,5,1
5,Oceania,1142110,15547.211111,6,5


Sebuah multinational company lain memiliki pendekatan strategi yang lain. Bukan dengan pendekatan benua prioritas target pasar, melainkan langsung menyasar negara prioritas. Perusahaan ini juga tidak menekankan kuantitas pasar (market size), tapi lebih ke daya beli pasar (consumer's buying power).  
Salah satu aspek yang mencerminkan daya beli pasar di suatu negara adalah GNP. Asumsinya semakin tinggi GNP suatu negara, maka semakin prioritas menjadi target product marketing. Dengan hanya menyasar negara-negara yang menyumbang 80 persen dari total GNP Global, maka operasi pemasaran akan lebih efisien.  
Perusahaan menugaskan Anda untuk mendapatkan output table seperti di bawah ini dari World Database. Tulis SQL query untuk menampilkan:  
* Nama negara prioritas,
* Persentase GNP tiap negara dibandingkan total GNP Global,
* Nilai kumulatif (Moving Sum) dari persentase GNP hanya sampai 80 persen saja,
* Peringkat berdasarkan persentase GNP, dan
* Kelompokkan negara-negara prioritas ini ke dalam 4 kelompok Market Priority!
* Tampilkan hanya negara-negara yang GNP-nya di atas satu persen!

,Country_Name,GNP_Percentage,Cumulative_GNP,GNP_Rank,Market_Priority_1234
0,United States,28.99,28.99,1,1
1,Japan,12.90,41.89,2,1
2,Germany,7.27,49.16,3,1
3,France,4.85,54.01,4,1
4,United Kingdom,4.70,58.71,5,2
5,Italy,3.96,62.67,6,2
6,China,3.35,66.02,7,2
7,Brazil,2.65,68.67,8,2
8,Canada,2.04,70.71,9,3
9,Spain,1.88,72.59,10,3


In [8]:
sql('''
WITH tabel1 AS (
SELECT country.name AS country_name, 
country.gnp*100/SUM(country.gnp) OVER() AS gnp_percentage 
FROM country
ORDER BY gnp_percentage DESC
),
tabel2 AS (SELECT *, SUM(gnp_percentage) OVER(ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS total_cumulative,
RANK() OVER(ORDER BY gnp_percentage DESC) AS gnp_rank
FROM tabel1)
SELECT *, NTILE(4) OVER(ORDER BY gnp_rank) AS quatile_cost 
FROM tabel2
WHERE gnp_percentage > 1

''')



,country_name,gnp_percentage,total_cumulative,gnp_rank,quatile_cost
0,United States,28.992426,28.992426,1,1
1,Japan,12.900882,41.893308,2,1
2,Germany,7.267497,49.160805,3,1
3,France,4.851948,54.012753,4,1
4,United Kingdom,4.695399,58.708152,5,2
5,Italy,3.957618,62.665770,6,2
6,China,3.346180,66.011950,7,2
7,Brazil,2.646028,68.657978,8,2
8,Canada,2.040075,70.698053,9,3
9,Spain,1.884635,72.582688,10,3
